https://docs.llamaindex.ai/en/stable/examples/node_parsers/semantic_chunking/

Semantic chunker, metni sabit bir yığın boyutuyla yığınlamak yerine, gömme benzerliğini kullanarak cümleler arasındaki kesme noktasını uyarlamalı olarak seçer. Bu, bir "chunk" 'ın anlamsal olarak birbiriyle ilişkili cümleler içermesini sağlar.

In [1]:
! pip install llama-index-embeddings-openai

   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB 991.0 kB/s eta 0:00:16
   ---------------------------------------- 0.1/15.4 MB 1.1 MB/s eta 0:00:15
   ---------------------------------------- 0.1/15.4 MB 1.1 MB/s eta 0:00:14
   ---------------------------------------- 0.1/15.4 MB 1.1 MB/s eta 0:00:14
   ---------------------------------------- 0.1/15.4 MB 610.6 kB/s eta 0:00:25
    --------------------------------------- 0.2/15.4 MB 737.3 kB/s eta 0:00:21
    --------------------------------------- 0.2/15.4 MB 623.6 kB/s eta 0:00:25
    --------------------------------------- 0.3/15.4 MB 684.6 kB/s eta 0:00:23
    --------------------------------------- 0.3/15.4 MB 710.0 kB/s eta 0:00:22
    --------------------------------------- 0.3/15.4 MB 710.0 kB/s eta 0:00:22
    --------------------------------------- 0.3/15.4 MB 634.7 kB/s eta 0:00:24
    --------------------------------------- 0.3/15.4 MB 634.7 kB/s 

# Setup Data

In [2]:
! wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'pg_essay.txt'

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
from PyPDF2 import PdfReader

# Bu fonksiyon, belirtilen PDF dosyasını açar, her sayfadaki metni çıkarır ve tüm metni birleştirerek tek bir metin dizesi olarak döndürür.
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        pdf_reader = PdfReader(f)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


pdf_path = "documents/PDF.pdf"
pdf_text = extract_text_from_pdf(pdf_path)


In [2]:
def write_text_to_file(text, file_path):
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)

pdf_path = "documents/PDF.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

output_file_path = "documents/PDF.txt"
write_text_to_file(pdf_text, output_file_path)


In [3]:
! pip install llama-index-readers-file

In [4]:
text = """Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.

Paris'in en büyüleyici yanlarından biri de, her köşede romantizminin hissedilmesidir. Gün batımında Seine Nehri'nde romantik bir tekne gezisi yapmak veya Versailles Bahçeleri'nde ay ışığında dolaşmak, Paris'in aşk dolu atmosferini keşfetmenin sadece birkaç yoludur. Tüm bu özellikler bir araya geldiğinde, Paris'in gerçekten de Fransa'nın kalbi olduğu açıkça ortaya çıkar.
"""

def write_text_to_file(text, file_path):
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)

output_file_path = "documents/fransa.txt"
write_text_to_file(text, output_file_path)


In [5]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["documents/PDF.txt"]).load_data()

c:\Users\havva\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
documents2 = SimpleDirectoryReader(input_files=["documents/fransa.txt"]).load_data()

# Define Semantic Splitter With Hugging Face

In [24]:
! pip install llama_index.embeddings.huggingface

   ---------------------------------------- 0.0/163.3 kB ? eta -:--:--
   ------- ------------------------------- 30.7/163.3 kB 660.6 kB/s eta 0:00:01
   ---------------------- ----------------- 92.2/163.3 kB 1.1 MB/s eta 0:00:01
   ------------------------------------- -- 153.6/163.3 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 163.3/163.3 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.6.0
    Uninstalling sentence-transformers-2.6.0:
      Successfully uninstalled sentence-transformers-2.6.0


In [7]:
from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
)

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [25]:
embed_model = HuggingFaceEmbedding()
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)

# also baseline splitter
base_splitter = SentenceSplitter(chunk_size=512, chunk_overlap = 0)

In [26]:
nodes = splitter.get_nodes_from_documents(documents)

In [17]:
len(nodes)

49

In [23]:
node_text = splitter.get_nodes_from_documents(documents2)

In [57]:
len(node_text)

2

# Inspecting the Chunks

In [32]:
print(nodes[1].get_content())

No compensation was provided to
the participants.
Participants
Experts
Recruitment aimed at including experts and health care
professionals experienced with exergame training of older adults
with mNCD, preferably (but not necessarily) with the exergame
training system “Senso (Flex)” or similar . For this purpose,
Dividat AG was asked to provide a contact list of 10 to 15
external experts and health care professionals with a variety in
age, sex, educational level, and experience in therap y of older
adults with mNCD, who are not emplo yed by Dividat AG or
had recei ved any funds from Dividat AG for their work. All
recommended experts and health care professionals were
contacted via email between November and December 2020.
By applying broad inclusion criteria, a rich spectrum of experts
and health care professionals were considered in the study ,
which in turn will foster the usability of the resulting program
in clinical practice. The specif ic eligibility criteria comprised
the follow

In [33]:
print(nodes[2].get_content())

Suitable patients were identif ied from
medical records and patient registries of memory clinics or from
diagnostics that had just been performed. Interested patients
were contacted by the research team by telephone or email to
clarify or obtain further information about the study procedures
and to register interest in participating in the study .
Subsequently , all patients were fully informed about the study
procedures in a face-to-f ace meeting at the patient’ s homes. In
addition, patients of interest were screened for eligibility . The
eligibility criteria are presented in Textbox 1.
JMIR Serious Games 2023 | vol. 


In [34]:
print(nodes[3].get_content())

11 | e37616 | p. 3 https://g ames.jmir .org/2023/1/e37616
(page number not for citation purposes)Manser et al JMIR SERIOUS GAMES
XSL•FO
RenderXTextbox 1. Description of all eligibility criteria.
• Inclusion criteria
• Participants fulfilling all the following inclusion criteria were eligible:
• (1= mild neurocogniti ve disorder [mNCD]) clinical diagnosis of “mNCD” according to International Classif ication of Diseases 11th
Revision [7] or Diagnostic and Statistical Manual of Mental Disorders 5th Edition [9]) OR (2=sMCI). Patients “screened for MCI”
according to the following criteria: (1) informant (ie, health care professional)–based suspicion of mild cogniti ve impairment confirmed
by (2) an objecti ve screening of mild cogniti ve impairment based on the German version of the using the Quick Mild Cogniti ve
Impairment Screen [55] with (b1) a recommended cutof f score for cogniti ve impairment (mild cogniti ve impairment or dementia) of
<62/100 [56], while (b2) not falling below the c

In [61]:
print(node_text[0].get_content())

Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. 


In [60]:
print(node_text[1].get_content())

Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.

Paris'in en büyüleyici yanlarından biri de, her köşede romantizminin hissedilmesidir. Gün batımında Seine Nehri'nde romantik bir tekne gezisi yapmak veya Versailles Bahçeleri'nde ay ışığında dolaşmak, Paris'in aşk dolu atmosferini keşfetmenin sadece birkaç yoludur. Tüm bu özellikler bir araya geldiğinde, Paris'in gerçekten de Fransa'nın kalbi olduğu açıkça ortaya çıkar.



# Compare against Baseline

In [35]:
base_nodes = base_splitter.get_nodes_from_documents(documents)

In [64]:
len(base_nodes)

116

In [36]:
print(base_nodes[2].get_content())

Conclusions: The psychosocial consequences of patients’ self-percei ved cogniti ve deterioration might be more burdensome
than the cogniti ve changes themselv es. Older adults with mNCD prefer integrative forms of training (such as exergaming) and
are primarily motivated by enjoyment or fun in exercising and the effectiveness of the training. Putting the synthesized perspecti ves
of training goals, settings, and requirements for exergames and training components into conte xt, our considerations point to
opportunities for impro vement in research and rehabilitation, either by adapting existing exergames to patients with mNCDs or
by developing novel exergames and exergame-based training concepts specif ically tailored to meet patient requirements and
needs.
(JMIR Serious Games 2023;11:e37616) doi: 10.2196/37616
KEYW ORDS
cognition; exercise; exergame; design; development; neurosciences; technology; training
JMIR Serious Games 2023 | vol. 11 | e37616 | p. 1 https://g ames.jmir .org/2023/

In [62]:
base_nodes_text = base_splitter.get_nodes_from_documents(documents2)

In [63]:
len(base_nodes_text)

1

In [68]:
print(base_nodes_text[0].get_content())

Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.

Paris'in en büyüleyici yanlarından biri de, her köşede romantizminin hissedilmesidir. Gün batımında Seine Nehri'nde romantik bir tekne gezisi yapmak veya 

# SentenceSplitter

In [71]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter()

nodes_parse = parser.get_nodes_from_documents(documents)

In [72]:
len(nodes_parse)

44

In [73]:
print(nodes_parse[0].get_content())

Original Paper
Design Consider ations for an Exergame-Based Training
Intervention for Older Adults With Mild Neurocognitiv e Disorder :
Qualitativ e Study Including Focus Groups With Exper ts and Health
Care Professionals and Individual Semistr uctured In-depth Patient
Interviews
Patrick Manser1, MSc; Manuela Adcock-Omlin1, DSc; Eling D de Bruin1,2,3, PhD
1Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and Technology , ETH Zurich,
Zurich, Switzerland
2Division of Physiotherap y, Department of Neurobiology , Care Sciences and Society , Karolinska Institutet, Stockholm, Sweden
3Department of Health, OST – Eastern Swiss University of Applied Sciences, St.Gallen, Switzerland
Corr esponding Author:
Patrick Manser , MSc
Motor Control and Learning Group – Institute of Human Movement Sciences and Sport, Department of Health Sciences and
Technology , ETH Zurich
Leopold-Ruzicka-W eg 4, CH-8093 Zurich, Switzerland
Zurich, CH-8093
S

In [74]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter()

nodes_parse_text = parser.get_nodes_from_documents(documents2)

In [75]:
len(nodes_parse_text)

1

In [76]:
print(nodes_parse_text[0].get_content())

Paris: Fransa'nın Kalbi

Paris, Fransa'nın başkenti ve en büyük şehri olarak sadece bir yer değil, aynı zamanda Fransız kültürünün ve tarihinin kalbidir. Seine Nehri'nin kıyısında yer alan bu büyüleyici şehir, muhteşem mimarisi, sanat eserleri ve romantizmiyle dünya genelinde büyük ilgi görüyor. Efsanevi Eiffel Kulesi, tarihi Notre-Dame Katedrali ve Louvre Müzesi gibi simge yapılar, Paris'i benzersiz kılan unsurlardan sadece birkaçıdır.

Paris'in güzellikleri sadece yapılarla sınırlı değildir; şehrin sokaklarındaki atmosfer de benzersizdir. Montmartre gibi bohem bölgelerde dolaşmak, Champs-Élysées'deki lüks mağazalarda alışveriş yapmak veya Seine'in kıyısında romantik bir yürüyüş yapmak, Paris'in sunduğu çeşitli deneyimlerden sadece birkaçıdır. Ayrıca, şehrin zengin gastronomi sahnesi, dünya mutfağının en lezzetli örneklerini sunar.

Paris'in en büyüleyici yanlarından biri de, her köşede romantizminin hissedilmesidir. Gün batımında Seine Nehri'nde romantik bir tekne gezisi yapmak veya 

# Setup Query Engine

In [47]:
! pip install llama-index

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
    --------------------------------------- 0.0/2.0 MB 991.0 kB/s eta 0:00:02
   - -------------------------------------- 0.1/2.0 MB 812.7 kB/s eta 0:00:03
   - -------------------------------------- 0.1/2.0 MB 762.6 kB/s eta 0:00:03
   - -------------------------------------- 0.1/2.0 MB 762.6 kB/s eta 0:00:03
   - -------------------------------------- 0.1/2.0 MB 374.1 kB/s eta 0:00:06
   - -------------------------------------- 0.1/2.0 MB 374.1 kB/s eta 0:00:06
   --- ------------------------------------ 0.2/2.0 MB 490.7 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.0 MB 490.7 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.0 MB 436.9 kB/s eta 0:00:05
   --- ------------------------------------ 0.2/2.0 MB 420.8 kB/s eta 0:00:05
   ---- ----------------------------------- 0.2/2.0 MB 402.0 kB/s eta 0:00:05
   ---- ----------------------------------- 0.2/2.0 MB 405.0 kB/s eta 0:0

In [48]:
from llama_index.core import VectorStoreIndex
from llama_index.core.response.notebook_utils import display_source_node

In [ ]:
vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine()

In [ ]:
base_vector_index = VectorStoreIndex(base_nodes)
base_query_engine = base_vector_index.as_query_engine()